In [1]:
import tensorflow as tf
from training_utils import download_file, get_batches, read_and_decode_single_example, load_validation_data, \
    download_data, evaluate_model, get_training_data, load_weights, flatten

C:\Users\eric\Anaconda2\envs\exts-aml2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
config = tf.ConfigProto()
how = "normal"
which = 8
batch_size = 32
dataset = 8
model_name = "model_s1.0.0.29l.8.2"
## Evaluate on test data multi-class
with tf.Session() as sess:
    # restore the graph
    saver = tf.train.import_meta_graph('./model/' + model_name + '.ckpt.meta', clear_devices=True)
    
    # restore the weights
    saver.restore(sess, './model/' + model_name + '.ckpt')
    
    # get the graph
    graph = tf.get_default_graph()
    
    # create the placeholders to feed
    X = tf.placeholder(shape=[None, 299, 299, 1], dtype=tf.float32)
    y = tf.placeholder(shape=[None], dtype=tf.int32)
    training = graph.get_tensor_by_name("is_training_2:0")
    
    # create the update ops
    extra_update_ops = tf.get_collection('extra_update_ops')
    probabilities = graph.get_tensor_by_name("probabilities:0")
    predictions = tf.argmax(probabilities, axis=1, output_type=tf.int64)
    recall, rec_op = tf.metrics.recall(labels=y, predictions=predictions, updates_collections=tf.GraphKeys.UPDATE_OPS, name="recall")
    
    # get the accuracy
    accuracy, acc_op = tf.metrics.accuracy(
        labels=y,
        predictions=predictions,
        updates_collections=tf.GraphKeys.UPDATE_OPS,
        name="accuracy",
    )
    sess.run(tf.local_variables_initializer())
    
    print("Restoring model", model_name)
    
    # load the test data
    X_te, y_te = load_validation_data(how=how, data="test", which=dataset)
    print("Evaluating on test data")
    
    test_accuracy = []
    test_recall = []
    test_predictions = []
    ground_truth = []
    test_recall_value = 0
    i = 1
    for X_batch, y_batch in get_batches(X_te, y_te, batch_size, distort=False):
        print("Batch", i)
        i += 1
        _, yhat, test_acc_value = sess.run([extra_update_ops, predictions, accuracy], feed_dict=
        {
            X: X_batch,
            y: y_batch,
            training: False,
        })

        test_accuracy.append(test_acc_value)
        test_recall.append(test_recall_value)
        test_predictions.append(yhat)
        ground_truth.append(y_batch)

    # print the results
    print()
    print("Mean Test Accuracy:", np.mean(test_accuracy))
    print("Mean Test Recall:", np.mean(test_recall))
    
    # unlist the predictions and truth
    test_predictions = flatten(test_predictions)
    ground_truth = flatten(ground_truth)
    
    # save the predictions and truth for review
    np.save(os.path.join("data", "predictions_" + model_name + ".npy"), test_predictions)
    np.save(os.path.join("data", "truth_" + model_name + ".npy"), ground_truth)

INFO:tensorflow:Restoring parameters from ./model/model_s1.0.0.29l.8.2.ckpt
Restoring model model_s1.0.0.29l.8.2
Evaluating on test data
Batch 1
